In [0]:
!pip install langdetect

In [0]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import random
from os.path import join, dirname

In [0]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
import string
import nltk
from nltk import PorterStemmer
import re
nltk.download('stopwords')
nltk.download('wordnet')
stopwords = nltk.corpus.stopwords.words('english')
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk import sent_tokenize, word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
from gensim import corpora
import gensim
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
from gensim.sklearn_api import W2VTransformer
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

In [0]:
#Collecting input from the URL given
URL_training  = 'https://raw.githubusercontent.com/thiagorainmaker77/liar_dataset/master/train.tsv'
URL_validation  = 'https://raw.githubusercontent.com/thiagorainmaker77/liar_dataset/master/valid.tsv'
URL_test  = 'https://raw.githubusercontent.com/thiagorainmaker77/liar_dataset/master/test.tsv'

    


In [0]:
df_training = pd.read_table(URL_training,
                             names = ['id',	'label'	,'statement',	'subject',	'speaker', 	'job', 	'state',	'party',	'barely_true_c',	'false_c',	'half_true_c',	'mostly_true_c',	'pants_on_fire_c',	'venue'])

    
df_validation = pd.read_table(URL_validation,
                             names =['id',	'label'	,'statement',	'subject',	'speaker', 	'job', 	'state',	'party',	'barely_true_c',	'false_c',	'half_true_c',	'mostly_true_c',	'pants_on_fire_c',	'venue'])


df_test = pd.read_csv(URL_test, sep='\t', 
                            names =['id',	'label'	,'statement',	'subject',	'speaker', 	'job', 	'state',	'party',	'barely_true_c',	'false_c',	'half_true_c',	'mostly_true_c',	'pants_on_fire_c',	'venue'])

In [0]:
df_training.loc[0,:]

id                                                         2635.json
label                                                          false
statement          Says the Annies List political group supports ...
subject                                                     abortion
speaker                                                 dwayne-bohac
job                                             State representative
state                                                          Texas
party                                                     republican
barely_true_c                                                      0
false_c                                                            1
half_true_c                                                        0
mostly_true_c                                                      0
pants_on_fire_c                                                    0
venue                                                       a mailer
Name: 0, dtype: object

In [0]:
df_training

,id,label,statement,subject,speaker,job,state,party,barely_true_c,false_c,half_true_c,mostly_true_c,pants_on_fire_c,venue
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN
5,12465.json,true,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece
6,2342.json,barely-true,Jim Dunnam has not lived in the district he re...,candidates-biography,republican-party-texas,NaN,Texas,republican,3.0,1.0,1.0,3.0,1.0,a press release.
7,153.json,half-true,I'm the only person on this stage who has work...,ethics,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,"a Democratic debate in Philadelphia, Pa."
8,5602.json,half-true,"However, it took $19.5 million in Oregon Lotte...",jobs,oregon-lottery,NaN,NaN,organization,0.0,0.0,1.0,0.0,1.0,a website
9,9741.json,mostly-true,Says GOP primary opponents Glenn Grothman and ...,"energy,message-machine-2014,voting-record",duey-stroebel,State representative,Wisconsin,republican,0.0,0.0,0.0,1.0,0.0,an online video


In [0]:
df_training_statement = df_training.loc[:,"statement"]
df_training_statement

0        Says the Annies List political group supports ...
1        When did the decline of coal start? It started...
2        Hillary Clinton agrees with John McCain "by vo...
3        Health care reform legislation is likely to ma...
4        The economic turnaround started at the end of ...
5        The Chicago Bears have had more starting quart...
6        Jim Dunnam has not lived in the district he re...
7        I'm the only person on this stage who has work...
8        However, it took $19.5 million in Oregon Lotte...
9        Says GOP primary opponents Glenn Grothman and ...
10       For the first time in history, the share of th...
11       Since 2000, nearly 12 million Americans have s...
12       When Mitt Romney was governor of Massachusetts...
13       The economy bled $24 billion due to the govern...
14       Most of the (Affordable Care Act) has already ...
15       In this last election in November, ... 63 perc...
16       McCain opposed a requirement that the governme.

In [0]:
df_training_label = df_training.loc[:,"label"]
df_training_label

0              false
1          half-true
2        mostly-true
3              false
4          half-true
5               true
6        barely-true
7          half-true
8          half-true
9        mostly-true
10       mostly-true
11         half-true
12             false
13       mostly-true
14       barely-true
15         half-true
16              true
17       barely-true
18         half-true
19       mostly-true
20             false
21       mostly-true
22       mostly-true
23         half-true
24       barely-true
25             false
26       mostly-true
27         half-true
28             false
29       mostly-true
            ...     
10210           true
10211           true
10212      half-true
10213    barely-true
10214          false
10215    barely-true
10216          false
10217      half-true
10218          false
10219      half-true
10220    mostly-true
10221          false
10222           true
10223    mostly-true
10224    barely-true
10225      half-true
10226    most

In [0]:
df_validation_statement = df_validation.loc[:,"statement"]
df_validation_statement

0       We have less Americans working now than in the...
1       When Obama was sworn into office, he DID NOT u...
2       Says Having organizations parading as being so...
3          Says nearly half of Oregons children are poor.
4       On attacks by Republicans that various program...
5       Says when armed civilians stop mass shootings ...
6       Says Tennessee is providing millions of dollar...
7       The health care reform plan would set limits s...
8       Says Donald Trump started his career back in 1...
9       Bill White has a long history of trying to lim...
10      John McCains chief economic adviser during the...
11      Says 21,000 Wisconsin residents got jobs in 20...
12      State revenue projections have missed the mark...
13      The median income of a middle class family wen...
14      Every citizen is entitled to the freedom of sp...
15      Rick Perry has advocated abandoning Social Sec...
16      Two thirds to three quarters of people without...
17      Congre

In [0]:
df_validation_label = df_validation.loc[:,"label"]
df_validation_label

0       barely-true
1        pants-fire
2             false
3         half-true
4         half-true
5             false
6              true
7             false
8              true
9         half-true
10        half-true
11            false
12        half-true
13             true
14      barely-true
15        half-true
16        half-true
17      mostly-true
18      barely-true
19       pants-fire
20        half-true
21       pants-fire
22             true
23       pants-fire
24        half-true
25      barely-true
26      mostly-true
27       pants-fire
28      mostly-true
29       pants-fire
           ...     
1254          false
1255      half-true
1256    barely-true
1257          false
1258           true
1259     pants-fire
1260           true
1261          false
1262    mostly-true
1263           true
1264      half-true
1265           true
1266          false
1267          false
1268          false
1269          false
1270      half-true
1271          false
1272      half-true


In [0]:
df_test_statement = df_test.loc[:,"statement"]
df_test_statement

0       Building a wall on the U.S.-Mexico border will...
1       Wisconsin is on pace to double the number of l...
2       Says John McCain has done nothing to help the ...
3       Suzanne Bonamici supports a plan that will cut...
4       When asked by a reporter whether hes at the ce...
5       Over the past five years the federal governmen...
6       Says that Tennessee law requires that schools ...
7       Says Vice President Joe Biden "admits that the...
8       Donald Trump is against marriage equality. He ...
9       We know that more than half of Hillary Clinton...
10      We know there are more Democrats in Georgia th...
11      PolitiFact Texas says Congressman Edwards atta...
12             Denali is the Kenyan word for black power.
13      Says 57 percent of federal spending goes to th...
14           On residency requirements for public workers
15      Says the unemployment rate for college graduat...
16      Unfortunately we have documented instances whe...
17      A rece

In [0]:
df_test_label = df_test.loc[:,"label"]
df_test_label

0              true
1             false
2             false
3         half-true
4        pants-fire
5              true
6              true
7       barely-true
8              true
9       barely-true
10      barely-true
11      barely-true
12       pants-fire
13            false
14        half-true
15             true
16       pants-fire
17        half-true
18             true
19            false
20      mostly-true
21        half-true
22       pants-fire
23      mostly-true
24            false
25             true
26      barely-true
27        half-true
28             true
29      mostly-true
           ...     
1237    mostly-true
1238     pants-fire
1239     pants-fire
1240     pants-fire
1241           true
1242      half-true
1243    barely-true
1244    mostly-true
1245    barely-true
1246    barely-true
1247           true
1248    barely-true
1249    barely-true
1250      half-true
1251    mostly-true
1252    mostly-true
1253           true
1254    mostly-true
1255          false


In [0]:
# Training the model  
from sklearn.ensemble import BaggingClassifier as BRC
from sklearn.ensemble import ExtraTreesClassifier as ETC
pipeline = Pipeline([
        ('ngrams', TfidfVectorizer(ngram_range=(1, 1))),
        ('clf', BRC(base_estimator=ETC(n_estimators=30), n_estimators=100,bootstrap_features=True,oob_score=True,max_features = 7))
    ])
pipeline.fit(df_training_statement, df_training_label)

Pipeline(memory=None,
         steps=[('ngrams',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern=...
                                                                       min_impurity_split=None,
                                                                       min_samples_leaf=1,
                                                                       min_samples_split=2,
                       

In [0]:
predicted_labels = pipeline.predict(df_validation_statement)
predicted_labels

array(['half-true', 'half-true', 'half-true', ..., 'half-true',
       'half-true', 'half-true'], dtype=object)

In [0]:
accuracy = pipeline.score(df_validation_label,predicted_labels)
accuracy

0.9213395638629284

In [0]:
predicted_labels_test = pipeline.predict(df_test_statement)
predicted_labels_test

array(['half-true', 'mostly-true', 'half-true', ..., 'half-true',
       'half-true', 'half-true'], dtype=object)

In [0]:
accuracy_test = pipeline.score(df_test_label,predicted_labels_test)
accuracy_test

0.9218626677190213

In [0]:
# now solving the first task i.e classifying into two categories : true and false

# Converting six types of label to 2 types of label in Training Data

Binary_label = []
for row in df_training["label"]:
        if row == "true":
            Binary_label.append("true")
        elif row == "half-true":
            Binary_label.append("true")
        elif row == "barely-true":
            Binary_label.append("true")
        elif row == "mostly-true":
            Binary_label.append("true")
        elif row == "false":
            Binary_label.append("false")
        elif row == "mostly-false":
            Binary_label.append("false")    
        elif row == "pants-fire":
            Binary_label.append("false")
        else: Binary_label.append("NAN")
 
       
df_training["new_label"] = Binary_label

In [0]:
# Converting six types of label to 2 types of label in Validation Data

Binary_label_validation = []
for row in df_validation["label"]:
        if row == "true":
            Binary_label_validation.append("true")
        elif row == "half-true":
            Binary_label_validation.append("true")
        elif row == "barely-true":
            Binary_label_validation.append("true")
        elif row == "mostly-true":
            Binary_label_validation.append("true")
        elif row == "false":
            Binary_label_validation.append("false")
        elif row == "mostly-false":
            Binary_label_validation.append("false")    
        elif row == "pants-fire":
            Binary_label_validation.append("false")
        else: Binary_label_validation.append("NAN")
 
       
df_validation["new_label"] = Binary_label_validation

In [0]:
# Converting six types of label to 2 types of label in Test Data

Binary_label_test = []
for row in df_test["label"]:
        if row == "true":
            Binary_label_test.append("true")
        elif row == "half-true":
            Binary_label_test.append("true")
        elif row == "barely-true":
            Binary_label_test.append("true")
        elif row == "mostly-true":
            Binary_label_test.append("true")
        elif row == "false":
            Binary_label_test.append("false")
        elif row == "mostly-false":
            Binary_label_test.append("false")    
        elif row == "pants-fire":
            Binary_label_test.append("false")
        else: Binary_label_test.append("NAN")
 
       
df_test["new_label"] = Binary_label_test

In [0]:
df_training_new_label = df_training.loc[:,"new_label"]
df_validation_new_label = df_validation.loc[:,"new_label"]
df_test_new_label = df_test.loc[:,"new_label"]

In [0]:
pipeline_1 = Pipeline([
        ('ngrams', TfidfVectorizer(ngram_range=(1, 1))),
        ('clf', BRC(base_estimator=ETC(n_estimators=30), n_estimators=100,bootstrap_features=True,oob_score=True,max_features = 7))
    ])
pipeline_1.fit(df_training_statement, df_training_new_label)

Pipeline(memory=None,
         steps=[('ngrams',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern=...
                                                                       min_impurity_split=None,
                                                                       min_samples_leaf=1,
                                                                       min_samples_split=2,
                       

In [0]:
predicted_labels_1 = pipeline_1.predict(df_validation_statement)
predicted_labels_1

array(['true', 'true', 'true', ..., 'true', 'true', 'true'], dtype=object)

In [0]:
accuracy_1 = pipeline_1.score(df_validation_new_label,predicted_labels_1)
accuracy_1

1.0

In [0]:
predicted_labels_1_test = pipeline_1.predict(df_test_statement)
predicted_labels_1_test

array(['true', 'true', 'true', ..., 'true', 'true', 'true'], dtype=object)

In [0]:
accuracy_1_test = pipeline_1.score(df_test_new_label,predicted_labels_1_test)
accuracy_1_test

1.0